In [1]:
from typing import NamedTuple, Sequence, Tuple
import numba as nb
import numpy as np

# Common

In [2]:
FLOAT_TYPE = np.float64
FLOAT_MIN = np.finfo(FLOAT_TYPE).min
FLOAT_MAX = np.finfo(FLOAT_TYPE).max

class Point(NamedTuple):
    x: float
    y: float

class Vector(NamedTuple):
    x: float
    y: float

@nb.njit
def cross_product(u: Vector, v: Vector) -> float:
    return u.x * v.y - u.y * v.x

@nb.njit
def dot_product(u: Vector, v: Vector) -> float:
    return u.x * v.x + u.y * v.y

# Sutherland-Hodgman

In [3]:
@nb.njit
def _push(array: np.ndarray, n: int, value: Vector):
    array[n] = value
    return n + 1

@nb.njit
def _copy(src, dst, n):
    for i in range(n):
        dst[i] = src[i]

@nb.njit
def _inside(p: Point, r: Point, U: Vector):
    # U: a -> b direction vector
    # p is point r or s
    return U.x * (p.y - r.y) > U.y * (p.x - r.x)

@nb.njit
def _intersection(a: Point, V: Vector, r: Point, N: Vector):
    W = Vector(r.x - a.x, r.y - a.y)
    nw = dot_product(N, W)
    nv = dot_product(N, V)
    if nv != 0:
        t = nw / nv
        return True, Point(a.x + t * V.x, a.y + t * V.y)
    else:
        return False, Point(0.0, 0.0)

@nb.njit
def _polygon_area(polygon, length):
    area = 0.0
    a = Point(polygon[0][0], polygon[0][1])
    b = Point(polygon[1][0], polygon[1][1])
    U = Vector(b.x - a.x, b.y - a.y)
    for i in range(2, length):
        c = Point(polygon[i][0], polygon[i][1])
        V = Vector(a.x - c.x, a.y - c.y)
        area += abs(cross_product(U, V))
        b = c
        U = V
    return 0.5 * area

@nb.njit
def clip_polygons(polygon, clipper):
    n_output = len(polygon)
    n_clip = len(clipper)
    n_max = n_output + n_clip
    output = np.empty((n_max, 2), FLOAT_TYPE)
    subject = np.empty((n_max, 2), FLOAT_TYPE)

    # Copy polygon into output
    _copy(polygon, output, n_output)

    # Grab last point
    r = Point(clipper[n_clip - 1][0], clipper[n_clip - 1][1])
    for i in range(n_clip):
        s = Point(clipper[i][0], clipper[i][1])

        U = Vector(s.x - r.x, s.y - r.y)
        N = Vector(-U.y, U.x)
        if U.x == 0 and U.y == 0:
            continue

        # Copy output into subject
        length = n_output
        _copy(output, subject, length)
        # Reset
        n_output = 0
        # Grab last point
        a = Point(subject[length - 1][0], subject[length - 1][1])
        a_inside = _inside(a, r, U)
        for j in range(length):
            b = Point(subject[j][0], subject[j][1])

            V = Vector(b.x - a.x, b.y - a.y)
            if V.x == 0 and V.y == 0:
                continue

            b_inside = _inside(b, r, U)
            if b_inside:
                if not a_inside:  # out, or on the edge
                    succes, point = _intersection(a, V, r, N)
                    if succes:
                        n_output = _push(output, n_output, point)
                n_output = _push(output, n_output, b)
            elif a_inside:
                succes, point = _intersection(a, V, r, N)
                if succes:
                    n_output = _push(output, n_output, point)
                else:  # Floating point failure
                    b_inside = True  # flip it for consistency, will be set as a
                    n_output = _push(output, n_output, b)  # push b instead

            # Advance to next polygon edge
            a = b
            a_inside = b_inside

        # Exit early in case not enough vertices are left.
        if n_output < 3:
            return 0.0

        # Advance to next clipping edge
        r = s

    area = _polygon_area(output, n_output)
    return area

# Generate counter clockwise triangles

In [4]:
@nb.njit
def ccw(a):
    for i in range(len(a)):
        t = a[i]
        normal = (t[1][0] - t[0][0])*(t[2][1]-t[0][1])-(t[1][1]-t[0][1])*(t[2][0]-t[0][0])

        if normal < 0:
            a[i] = t[::-1]
     

a = np.random.rand(1_000_000, 3, 2)
b = np.random.rand(1_000_000, 3, 2)
ccw(a)
ccw(b)

In [5]:
def _area_of_intersection(a, b):
    n = len(a)
    out = np.zeros(n)
    for i in nb.prange(n):
        t0 = a[i]
        t1 = b[i]
        out[i] = clip_polygons(t0, t1)
    return out

In [6]:
area_of_intersection = nb.njit(_area_of_intersection)

In [7]:
%timeit area_of_intersection(a, b)

673 ms ± 3.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
parallel_area_of_intersection = nb.njit(_area_of_intersection, parallel=True)

In [9]:
%timeit parallel_area_of_intersection(a, b)

257 ms ± 4.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
